In [1]:
from models import vgg_c
import torch

## Model

In [2]:
model = vgg_c.vgg19_trans()
# print(model)
i = torch.rand((1, 3, 512, 512))
o, fetures = model(i)
print(o.size())
for feture in fetures:
    print(feture.size()) 

torch.Size([1, 1, 32, 32])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])


d:\Program\Conda\envs\py\lib\site-packages\torch\nn\functional.py:3253: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")


## Data

In [3]:
from datasets.crowd import Crowd
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
import os
import numpy as np
from losses.post_prob import Post_Prob
from losses.bay_loss import Bay_Loss

In [4]:
def train_collate(batch):
    transposed_batch = list(zip(*batch))
    images = torch.stack(transposed_batch[0], 0)
    points = transposed_batch[1]  # the number of points is not fixed, keep it as a list of tensor
    targets = transposed_batch[2]
    st_sizes = torch.FloatTensor(transposed_batch[3])
    return images, points, targets, st_sizes
post_prob = Post_Prob(
    sigma=8.0, 
    c_size=512, 
    stride=16, 
    background_ratio=0.15,
    use_background=True,
    device='cpu'
)
criterion = Bay_Loss(use_background=True, device='cpu')

In [5]:
datasets = {
    x : Crowd(os.path.join("UCF-Train-Val-Test", x), 
              crop_size=512, 
              downsample_ratio=16, 
              is_gray=False, method=x) 
    for x in ['train', 'val']
}
dataloaders = {
    x : DataLoader(datasets[x], 
                   collate_fn=(train_collate if x == 'train' else default_collate),
                   batch_size=(1 if x == 'train' else 1), 
                   shuffle=(True if x =='train' else False),
                   num_workers=0, 
                   pin_memory=(True if x == 'train' else False))
    for x in ['train', 'val']
}

## Train Epoch
测试各个部分数据的含义

In [8]:
for step, (inputs, points, targets, st_sizes) in enumerate(dataloaders['train']):
    # c, h, w = inputs.size()
    # inputs = inputs.view(-1, c, h, w)
    print('inputs.size() =', inputs.size())

    gd_count = np.array([len(p) for p in points], dtype=np.float32)
    points = [p for p in points]
    targets = [t for t in targets]
    for i, p in enumerate(points):
        print("points[{}].size() = {}".format(i, p.size()))

    outputs, features = model(inputs)
    # TODO prob_list  7
    print('st_sizes = ', st_sizes)
    prob_list = post_prob(points, st_sizes)
    for prob in prob_list:
        if prob != None:
            print('prob.size() =', prob.size())
        else :
            print("None")
    for target in targets:
        print('target.size() =',target.size())
        print('target =', target)
    print('outputs.size() =',outputs.size())
    loss = criterion(prob_list, targets, outputs)
    loss_c = 0
    for feature in features:
        mean_feature = torch.mean(feature, dim=0)
        mean_sum = torch.sum(mean_feature**2)**0.5
        cosine = 1 - torch.sum(feature*mean_feature, dim=1) / (mean_sum * torch.sum(feature**2, dim=1)**0.5 + 1e-5)
        loss_c += torch.sum(cosine)
    loss += loss_c

    N = inputs.size(0)
    pre_count = torch.sum(outputs.view(N, -1), dim=1).detach().cpu().numpy()
    res = pre_count - gd_count
    if step == 0:
        break

inputs.size() = torch.Size([1, 3, 512, 512])
points[0].size() = torch.Size([76, 2])
st_sizes =  tensor([1580.])
prob.size() = torch.Size([77, 1024])
target.size() = torch.Size([76])
target = tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.7866, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.5316, 0.4770, 0.8145,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.5304, 0.4503,
        0.4978, 0.3279, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        0.5221, 1.0000, 1.0000, 1.0000, 0.5685, 0.9848, 1.0000, 1.0000, 1.0000,
        0.6059, 0.5815, 1.0000, 0.5961, 0.4556, 1.0000, 0.7179, 1.0000, 1.0000,
        1.0000, 0.5779, 1.0000, 0.9868, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000])
outputs.size() = torch.Size([1, 1, 32, 32])


d:\Program\Conda\envs\py\lib\site-packages\torch\nn\functional.py:3253: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")


## Loss

In [20]:
from math import ceil
ten = torch.rand((20))
print(ten)
print(ten[-1])
num = ceil(0.9 * (len(ten) - 1))
print(num)
topk = torch.topk(ten, num, largest=False)
print(topk)
print(topk[0])

tensor([0.0290, 0.0032, 0.4807, 0.4078, 0.8467, 0.5442, 0.1685, 0.7243, 0.6350,
        0.0663, 0.7532, 0.5711, 0.6797, 0.7012, 0.3887, 0.1317, 0.2648, 0.2201,
        0.9376, 0.6267])
tensor(0.6267)
18
torch.return_types.topk(
values=tensor([0.0032, 0.0290, 0.0663, 0.1317, 0.1685, 0.2201, 0.2648, 0.3887, 0.4078,
        0.4807, 0.5442, 0.5711, 0.6267, 0.6350, 0.6797, 0.7012, 0.7243, 0.7532]),
indices=tensor([ 1,  0,  9, 15,  6, 17, 16, 14,  3,  2,  5, 11, 19,  8, 12, 13,  7, 10]))
tensor([0.0032, 0.0290, 0.0663, 0.1317, 0.1685, 0.2201, 0.2648, 0.3887, 0.4078,
        0.4807, 0.5442, 0.5711, 0.6267, 0.6350, 0.6797, 0.7012, 0.7243, 0.7532])


In [24]:
cood = torch.arange(0, 512, 16, dtype=torch.float32, device='cpu') + 16 / 2 
print(cood)
print(cood.size())
cood.unsqueeze_(0)
print(cood)
print(cood.size())

tensor([  8.,  24.,  40.,  56.,  72.,  88., 104., 120., 136., 152., 168., 184.,
        200., 216., 232., 248., 264., 280., 296., 312., 328., 344., 360., 376.,
        392., 408., 424., 440., 456., 472., 488., 504.])
torch.Size([32])
tensor([[  8.,  24.,  40.,  56.,  72.,  88., 104., 120., 136., 152., 168., 184.,
         200., 216., 232., 248., 264., 280., 296., 312., 328., 344., 360., 376.,
         392., 408., 424., 440., 456., 472., 488., 504.]])
torch.Size([1, 32])


In [38]:
num_points_per_image = [len(points_per_image) for points_per_image in points]
print('num_points_per_image =', num_points_per_image)
all_points = torch.cat(points, dim=0)
print('all_points.size() =', all_points.size())
print(len(all_points))
x = all_points[:, 0].unsqueeze_(1)
y = all_points[:, 1].unsqueeze_(1)
print('x.size() =', x.size())
print('y.size() =', y.size())
x_dis = -2 * torch.matmul(x, cood) + x * x + cood * cood
y_dis = -2 * torch.matmul(y, cood) + y * y + cood * cood
print("x_dis.size() =", x_dis.size())
print("y_dis.size() =", y_dis.size())
x_dis.unsqueeze_(1)
y_dis.unsqueeze_(2)
print("After unsqueeze...")
print("x_dis.size() =", x_dis.size())
print("y_dis.size() =", y_dis.size())
dis = y_dis + x_dis
print("dis.size() =", dis.size())
dis = dis.view((dis.size(0), -1))
print("After view...")
print("dis.size() =", dis.size())
dis_list = torch.split(dis, num_points_per_image)
for i, dis_item in enumerate(dis_list):
    print(i, "dis_item.size() =", dis_item.size())
prob_list = []
softmax = torch.nn.Softmax(dim=0)
for dis, st_size in zip(dis_list, st_sizes):
    if len(dis) > 0:
        min_dis = torch.clamp(torch.min(dis, dim=0, keepdim=True)[0], min=0.0)
        d = st_size * 0.15
        bg_dis = (d - torch.sqrt(min_dis)) ** 2
        dis = -dis / (2.0 * 8.0 ** 2)
        prob = softmax(dis)
        print('prob.size() =', prob.size())

num_points_per_image = [9]
all_points.size() = torch.Size([9, 2])
9
x.size() = torch.Size([9, 1])
y.size() = torch.Size([9, 1])
x_dis.size() = torch.Size([9, 32])
y_dis.size() = torch.Size([9, 32])
After unsqueeze...
x_dis.size() = torch.Size([9, 1, 32])
y_dis.size() = torch.Size([9, 32, 1])
dis.size() = torch.Size([9, 32, 32])
After view...
dis.size() = torch.Size([9, 1024])
0 dis_item.size() = torch.Size([9, 1024])
prob.size() = torch.Size([9, 1024])
